In [47]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import numpy as np 
import torch_geometric.nn as pyg_nn 
import torch_geometric.utils as pyg_utils
from torch_geometric.data import Dataset, Data
import os.path as osp
import scipy.io as sio
from dataset import QUASARDataset

In [48]:
dir = '/Users/hankyang/Datasets/QUASAR'
dataset = QUASARDataset(dir)
data = dataset[0]
# X = torch.rand((5,5))
# idx = np.arange(0,4)

Processing...
Done!


In [49]:
def smat(x,n):
    X        = torch.zeros(n,n,dtype=torch.float64)
    si, sj   = torch.triu_indices(n,n)
    X[si,sj] = x 
    X        = X + X.t() # symmetric matrix
    return X

def mat(x,n):
    return x.view((n,n))

def blk_index(i,p):
    return np.arange(i*p, (i+1)*p)

def recover_X(vp,ep,ud_edges):
    N = vp.shape[0] # number of nodes
    n = 4*N # size of symmetric X
    X = torch.zeros((n,n),dtype=torch.float64)
    # assign diagonal blocks
    for i in range(N):
        idx = blk_index(i,4)
        X[idx,:][:,idx] = smat(vp[i,:],4)
    # assign off-diagonal blocks 
    # for edge_idx, edge in enumerate(ud_edges):
    #     blk   = smat(ep[edge_idx,:],4)
    #     nodei = edge[0]
    #     nodej = edge[1]
    #     idx_i = blk_index(nodei,4)
    #     idx_j = blk_index(nodej,4)

    #     X[idx_i,:][:,idx_j] = blk
    #     X[idx_j,:][:,idx_i] = blk
    return X

In [50]:
 
class ModelS(nn.Module):
    def __init__(self, mp_input_dim=6,
                       mp_hidden_dim=32,
                       mp_output_dim=64,
                       mp_num_layers=1, 
                       primal_node_mlp_hidden_dim=64,
                       primal_node_mlp_output_dim=10,
                       dual_node_mlp_hidden_dim=64,
                       dual_node_mlp_output_dim=10,
                       node_mlp_num_layers=1,
                       primal_edge_mlp_hidden_dim=64, 
                       primal_edge_mlp_output_dim=10, 
                       dual_edge_mlp_hidden_dim=64, 
                       dual_edge_mlp_output_dim=16, 
                       edge_mlp_num_layers=1, 
                       dropout_rate=0.2):
        super(ModelS,self).__init__()
        # Message passing
        self.mp_convs = nn.ModuleList()
        self.mp_convs.append(pyg_nn.SAGEConv(mp_input_dim,mp_hidden_dim))
        for i in range(mp_num_layers):
            self.mp_convs.append(pyg_nn.SAGEConv(mp_hidden_dim,mp_hidden_dim))
        self.mp_convs.append(pyg_nn.SAGEConv(mp_hidden_dim,mp_output_dim))

        # Post message passing
        # Primal node MLP
        self.primal_node_mlp = nn.ModuleList()
        self.primal_node_mlp.append(
            nn.Linear(mp_output_dim,primal_node_mlp_hidden_dim,dtype=torch.float64))
        for i in range(node_mlp_num_layers):
            self.primal_node_mlp.append(
                nn.Linear(primal_node_mlp_hidden_dim,primal_node_mlp_hidden_dim,dtype=torch.float64))
        self.primal_node_mlp.append(
            nn.Linear(primal_node_mlp_hidden_dim,primal_node_mlp_output_dim,dtype=torch.float64))
        # Dual node MLP
        self.dual_node_mlp = nn.ModuleList()
        self.dual_node_mlp.append(
            nn.Linear(mp_output_dim,dual_node_mlp_hidden_dim,dtype=torch.float64))
        for i in range(node_mlp_num_layers):
            self.dual_node_mlp.append(
                nn.Linear(dual_node_mlp_hidden_dim,dual_node_mlp_hidden_dim,dtype=torch.float64))
        self.dual_node_mlp.append(
            nn.Linear(dual_node_mlp_hidden_dim,dual_node_mlp_output_dim,dtype=torch.float64))
        # Primal edge MLP
        self.primal_edge_mlp = nn.ModuleList()
        self.primal_edge_mlp.append(
            nn.Linear(mp_output_dim,primal_edge_mlp_hidden_dim,dtype=torch.float64))
        for i in range(edge_mlp_num_layers):
            self.primal_edge_mlp.append(
                nn.Linear(primal_edge_mlp_hidden_dim,primal_edge_mlp_hidden_dim,dtype=torch.float64))
        self.primal_edge_mlp.append(
            nn.Linear(primal_edge_mlp_hidden_dim,primal_edge_mlp_output_dim,dtype=torch.float64))
        # Dual edge MLP
        self.dual_edge_mlp = nn.ModuleList()
        self.dual_edge_mlp.append(
            nn.Linear(mp_output_dim,dual_edge_mlp_hidden_dim,dtype=torch.float64))
        for i in range(edge_mlp_num_layers):
            self.dual_edge_mlp.append(
                nn.Linear(dual_edge_mlp_hidden_dim,dual_edge_mlp_hidden_dim,dtype=torch.float64))
        self.dual_edge_mlp.append(
            nn.Linear(dual_edge_mlp_hidden_dim,dual_edge_mlp_output_dim,dtype=torch.float64))
        self.dropout_rate = dropout_rate

    def forward(self,data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        print(x.type())
        num_nodes = data.num_nodes
        ud_edges  = data.ud_edges
        # Message passing
        for mp_layer in self.mp_convs:
            x = mp_layer(x,edge_index)
            x = F.relu(x)
            x = F.dropout(x,p=self.dropout_rate,training=self.training)
        
        # Post message passing
        # Primal node
        vp = []
        for i in range(num_nodes):
            xi = x[i,:] # feature of i-th node
            for mlp_layer in self.primal_node_mlp:
                xi = mlp_layer(xi)
                xi = F.relu(xi)
                xi = F.dropout(xi,p=self.dropout_rate,training=self.training)
            vp.append(xi)
        vp = torch.stack(vp) # num_nodes x primal_node_mlp_output_dim
        # Dual node
        vd = []
        for i in range(num_nodes):
            xi = x[i,:]
            for mlp_layer in self.dual_node_mlp:
                xi = mlp_layer(xi)
                xi = F.relu(xi)
                xi = F.dropout(xi,p=self.dropout_rate,training=self.training)
            vd.append(xi)
        vd = torch.stack(vd) # num_nodes x dual_node_mlp_output_dim
        # Primal edge
        ep = []
        for edge in ud_edges:
            xi  = x[edge[0],:]
            xj  = x[edge[1],:]
            xij = xi + xj
            for mlp_layer in self.primal_edge_mlp:
                xij = mlp_layer(xij)
                xij = F.relu(xij)
                xij = F.dropout(xij,p=self.dropout_rate,training=self.training)
            ep.append(xij)
        ep = torch.stack(ep)
        # Dual edge
        ed = []
        for edge in ud_edges:
            xi  = x[edge[0],:]
            xj  = x[edge[1],:]
            xij = xi + xj
            for mlp_layer in self.dual_edge_mlp:
                xij = mlp_layer(xij)
                xij = F.relu(xij)
                xij = F.dropout(xij,p=self.dropout_rate,training=self.training)
            ed.append(xij)
        ed = torch.stack(ed)

        return x, vp, vd, ep, ed

In [51]:
# Test evaluate model
model = ModelS()
model.double()
model.eval()
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name)
#         print(param.data)
x, vp, vd, ep, ed = model(data)
# print(smat(vp[0,:],4))
# X = torch.zeros((84,84))
# X[0:4,:][:,0:4] = smat(vp[0,:],4)
# X[4:8,:][:,4:8] = smat(vp[1,:],4)
X = recover_X(vp,ep,data.ud_edges)


torch.DoubleTensor
